In [41]:
import os 
os.chdir("..")
import numpy as np
import sympy
import cirq
import tensorflow_quantum as tfq
from tqdm import tqdm
import tensorflow as tf

from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

%load_ext autoreload
%autoreload 2


n_qubits = 5
qlr = 0.01
qepochs = 10**3
verbose=0
g=1
J=0
noise=0.0
problem="TFIM"
vqe_handler = VQE(n_qubits=n_qubits, lr=qlr, epochs=qepochs, patience=100,
                  random_perturbations=True, verbose=verbose, g=g, J = J, noise=noise, problem=problem)

iid = IdInserter(n_qubits=n_qubits)
Simp = Simplifier(n_qubits=n_qubits)
killer = UnitaryMurder(vqe_handler)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [160]:
#indexed_circuit = np.random.choice(range(vqe_handler.number_of_cnots+2*vqe_handler.n_qubits), 20)
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]
for k in range(vqe_handler.n_qubits):
    indexed_circuit.append(k+vqe_handler.number_of_cnots)


In [ ]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(circuit_input, symbol_names=symbols, operators=tfq.convert_to_tensor([vqe_handler.observable]*1000),
        initializer=tf.keras.initializers.RandomUniform(minval=-np.pi, maxval=np.pi))

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=vqe_handler.lr)
model.compile(optimizer=adam, loss='mse')
tfqcircuit = tfq.convert_to_tensor(cbatch)
qoutput = tf.ones((10**3, 1))*vqe_handler.lower_bound_Eg
from datetime import datetime
now = datetime.now()
h=model.fit(x=tfqcircuit, y=qoutput, batch_size=10**3, epochs=100,
          verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=vqe_handler.patience, mode="min", min_delta=10**-3)])
print(datetime.now()-now)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(h.history["loss"])